In [53]:
%load_ext autoreload
%autoreload 2

# remove warnings
import warnings
warnings.filterwarnings('ignore')
# ---

import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
import numpy as np

from lib.graph_init import *
from lib.hard_hfs import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import Data

In [54]:
train = pd.read_csv('../pickles/HOG_train.csv', sep = ",", header = None)
trainOutput = pd.read_csv('../data/Ytr.csv', sep = ",")
test = pd.read_csv('../pickles/HOG_test.csv', sep = ",", header = None)

In [55]:
train.drop(train.columns[len(train.columns)-1], axis=1, inplace=True)
test.drop(test.columns[len(test.columns)-1], axis=1, inplace=True)

In [56]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.985244,0.822128,0.909710,0.665221,1.113231,0.809004,0.943652,0.696333,0.882468,0.979157,0.980727,1.202559,0.777422,1.042495,0.873644,1.293949,0.848496,0.894750,1.071137,1.132274,1.359492,0.848587,1.086852,1.002670,1.565936,0.979042,0.951022,1.158886,1.294948,1.552957,0.939612,1.160195,1.127392,1.834191,1.073831,1.029878,1.161847,1.319926,1.706460,1.041365,1.233406,1.073707,1.925199,1.297929,1.121277,1.116525,1.178654,1.562564,1.071630,1.221451,...,1.026946,1.119953,0.874096,1.349081,1.168928,1.103513,1.028646,1.163638,1.774966,1.100823,1.205427,0.942903,1.504311,1.310693,1.184658,1.112178,1.312044,1.907034,1.079935,1.297422,1.013499,1.649553,1.307407,1.160979,1.199799,1.255425,1.784976,1.013655,1.261681,1.058975,1.514425,1.142951,1.111103,1.115060,1.097585,1.406703,0.890349,1.180977,0.961679,1.247913,0.908045,1.021145,1.116415,0.860579,0.892667,0.728771,1.244633,0.944710,0.856066,0.689813
std,1.472312,1.251851,1.607435,1.088982,1.819687,1.196295,1.672962,1.117677,1.360720,1.390962,1.485733,2.048414,1.390150,1.656807,1.330221,2.217912,1.409916,1.404908,1.506442,1.573318,2.054496,1.303891,1.617173,1.439548,2.430584,1.513530,1.380749,1.640294,1.755418,2.292926,1.330725,1.665471,1.519076,2.683237,1.546842,1.428312,1.596603,1.795989,2.392224,1.420877,1.781973,1.479133,2.701812,1.782662,1.574473,1.527765,1.689688,2.251170,1.477344,1.674445,...,1.452373,1.717531,1.350446,2.138431,1.656945,1.500399,1.417081,1.723257,2.633201,1.534923,1.753533,1.330009,2.183828,1.808646,1.596867,1.517411,1.793396,2.843647,1.476730,1.810478,1.392805,2.313467,1.762781,1.579604,1.578440,1.740374,2.740114,1.457264,1.699129,1.320273,2.148051,1.638240,1.526475,1.499216,1.639374,2.341417,1.323341,1.642356,1.287900,1.956266,1.403798,1.402747,1.532283,1.293593,1.557437,1.072328,1.741641,1.314707,1.384136,1.080549
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [57]:
combined = train.append(test)
combined.reset_index(inplace=True)
combined.drop('index',inplace=True,axis=1)

## HFS 
0 is unlabeled, but was a class: that's why we use the +1/-1 trick

In [59]:
input_vect = combined
target_vect = np.array(trainOutput['Prediction'])
target_vect = np.append(target_vect+1,[0 for i in range(len(combined) - len(target_vect))])
K = K_poly

In [60]:
length = 5000

ttrain, tval, tsorter = get_train_val(train[:length], 5/7)
input_vect = train[:length]
target_vect = (trainOutput[:length]['Prediction']+1)*(1-tsorter)

K = K_poly[:length].T[:length]

In [61]:
lp = LaplacianParams()

# sim = build_graph(input_vect, GraphParams())
sim = K

print(sim)

[[ 1.          0.02982342  0.05895453 ...,  0.01453353  0.0236294
   0.04572605]
 [ 0.02982342  1.          0.03267616 ...,  0.00412909  0.0130808   0.02639   ]
 [ 0.05895453  0.03267616  1.         ...,  0.00988324  0.01960461
   0.05783863]
 ..., 
 [ 0.01453353  0.00412909  0.00988324 ...,  1.          0.00234235
   0.00612289]
 [ 0.0236294   0.0130808   0.01960461 ...,  0.00234235  1.          0.01535978]
 [ 0.04572605  0.02639     0.05783863 ...,  0.00612289  0.01535978  1.        ]]


In [62]:
L = build_laplacian(sim,lp)

print(L.shape)

(5000, 5000)


In [63]:
hfs0, confidence = simple_hfs(input_vect, target_vect, L, sim)
hfs0.shape

(5000,)

In [64]:
# IdTest = np.array([i for i in range(1, 1 + len(combined) - len(train))])
# output = [int(x) for x in hfs0[IdTest + len(train) - 1] - 1]
# df_output = pd.DataFrame()
# df_output['Id'] = IdTest
# df_output['Prediction'] = output
# df_output[['Id','Prediction']].to_csv('../predictions/test_hfs_K_poly.csv', sep = ",", index=False)

In [65]:
output = hfs0[tsorter[tsorter == 1].index] - 1